# Лабораторная работа №4 (Проведение исследований со случайным лесом)

# 2. Создание бейзлайна и оценка качества
**2.a** Обучить модели из `sklearn` (для классификации и регрессии) для выбранных наборов данных  
**2.b** Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных 


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Случайный лес из sklearn
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             mean_squared_error, mean_absolute_error, r2_score)
from sklearn.model_selection import GridSearchCV

In [10]:
# 1. Загрузка и первичная обработка данных
data = pd.read_csv('student_data.csv', sep=',')
print("Shape:", data.shape)

# (Опционально) удаляем пропуски, если есть
data.dropna(inplace=True)

# Бинарная классификация: "сдал/не сдал"
data_class = data.copy()
data_class['passed'] = (data_class['G3'] >= 10).astype(int)

# Признаки для классификации (простейший вариант)
X_class = data_class[['G1', 'G2', 'studytime', 'failures', 'absences']]
y_class = data_class['passed']

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Бейзлайн (Classification) - RandomForest без особых настроек
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(Xc_train, yc_train)
yc_pred = rf_clf.predict(Xc_test)

acc = accuracy_score(yc_test, yc_pred)
prec = precision_score(yc_test, yc_pred)
rec = recall_score(yc_test, yc_pred)
f1 = f1_score(yc_test, yc_pred)

print("Бейзлайн RandomForest (Classification):")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

# Регрессия: предсказываем G3
X_reg = data[['G1', 'G2', 'studytime', 'failures', 'absences']]
y_reg = data['G3']

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Бейзлайн (Regression) - RandomForest без особых настроек
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(Xr_train, yr_train)
yr_pred = rf_reg.predict(Xr_test)

mse = mean_squared_error(yr_test, yr_pred)
mae = mean_absolute_error(yr_test, yr_pred)
r2 = r2_score(yr_test, yr_pred)

print("\nБейзлайн RandomForest (Regression):")
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R^2:  {r2:.4f}")


Shape: (395, 33)
Бейзлайн RandomForest (Classification):
Accuracy:  0.8734
Precision: 0.9388
Recall:    0.8679
F1-score:  0.9020

Бейзлайн RandomForest (Regression):
MSE:  2.6369
MAE:  1.0548
R^2:  0.8714


**Вывод**:
- Базовый случайный лес для классификации показал некоторые метрики (accuracy, f1).  
- Базовый случайный лес для регрессии дал значения (MSE, MAE, R^2).  

# 3. Улучшение бейзлайна
**3.a** Сформулировать гипотезы (предобработка данных, визуализация, новые признаки, подбор гиперпараметров...)  
**3.b** Проверить гипотезы  
**3.c** Сформировать улучшенный бейзлайн  
**3.d** Обучить модели с улучшенным бейзлайном (для классификации и регрессии)  
**3.e** Оценить качество моделей  
**3.f** Сравнить результаты с пунктом 2  
**3.g** Сделать выводы

In [11]:
# Пример гипотез:
# - Добавить признак total_G1_G2 = G1 + G2
# - Удалить выбросы, например, absences > 40
# - Подобрать гиперпараметры n_estimators, max_depth, min_samples_leaf, max_features и т.д.

# 1) Удаление выбросов (пример)
data_improved = data.copy()
data_improved = data_improved[data_improved['absences'] <= 40]

# 2) Создание нового признака
data_improved['total_G1_G2'] = data_improved['G1'] + data_improved['G2']

# ---------- КЛАССИФИКАЦИЯ ----------
data_class_improved = data_improved.copy()
data_class_improved['passed'] = (data_class_improved['G3'] >= 10).astype(int)
X_class2 = data_class_improved[['G1','G2','studytime','failures','absences','total_G1_G2']]
y_class2 = data_class_improved['passed']

Xc2_train, Xc2_test, yc2_train, yc2_test = train_test_split(
    X_class2, y_class2, test_size=0.2, random_state=42, stratify=y_class2
)

# Пробуем подобрать гиперпараметры
param_grid_clf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', None]
}

rf_clf_cv = GridSearchCV(RandomForestClassifier(random_state=42),
                         param_grid_clf, cv=5, scoring='f1', n_jobs=-1)
rf_clf_cv.fit(Xc2_train, yc2_train)

best_clf = rf_clf_cv.best_estimator_
yc2_pred_best = best_clf.predict(Xc2_test)

acc_best = accuracy_score(yc2_test, yc2_pred_best)
prec_best = precision_score(yc2_test, yc2_pred_best)
rec_best = recall_score(yc2_test, yc2_pred_best)
f1_best = f1_score(yc2_test, yc2_pred_best)

print("Улучшенный бейзлайн (RandomForest - Classification):")
print("Лучшие параметры:", rf_clf_cv.best_params_)
print(f"Accuracy:  {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall:    {rec_best:.4f}")
print(f"F1-score:  {f1_best:.4f}")

# ---------- РЕГРЕССИЯ ----------
X_reg2 = data_improved[['G1','G2','studytime','failures','absences','total_G1_G2']]
y_reg2 = data_improved['G3']

Xr2_train, Xr2_test, yr2_train, yr2_test = train_test_split(
    X_reg2, y_reg2, test_size=0.2, random_state=42
)

param_grid_reg = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt','log2', None]
}

rf_reg_cv = GridSearchCV(RandomForestRegressor(random_state=42),
                         param_grid_reg, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
rf_reg_cv.fit(Xr2_train, yr2_train)

best_reg = rf_reg_cv.best_estimator_
yr2_pred_best = best_reg.predict(Xr2_test)

mse_best = mean_squared_error(yr2_test, yr2_pred_best)
mae_best = mean_absolute_error(yr2_test, yr2_pred_best)
r2_best = r2_score(yr2_test, yr2_pred_best)

print("\nУлучшенный бейзлайн (RandomForest - Regression):")
print("Лучшие параметры:", rf_reg_cv.best_params_)
print(f"MSE:  {mse_best:.4f}")
print(f"MAE:  {mae_best:.4f}")
print(f"R^2:  {r2_best:.4f}")


Улучшенный бейзлайн (RandomForest - Classification):
Лучшие параметры: {'max_depth': 10, 'max_features': None, 'min_samples_leaf': 5, 'n_estimators': 100}
Accuracy:  0.9241
Precision: 0.9796
Recall:    0.9057
F1-score:  0.9412

Улучшенный бейзлайн (RandomForest - Regression):
Лучшие параметры: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 200}
MSE:  1.4459
MAE:  0.8484
R^2:  0.8966


**Выводы**:
- Удаление выбросов (`absences>40`) + добавление признака (`total_G1_G2`) + подбор гиперпараметров улучшили результаты модели (или нет).
- Можно расширять список гиперпараметров (например, `min_samples_split`, `bootstrap`, `oob_score=True` и т.д.).


# 4. Имплементация алгоритма машинного обучения(Random Forest)
**4.a** Самостоятельно имплементировать алгоритмы (для классификации и регрессии)  
**4.b** Обучить имплементированные модели (для классификации и регрессии)  
**4.c** Оценить качество имплементированных моделей  
**4.d** Сравнить результаты имплементированных моделей с пунктом 2  
**4.e** Сделать выводы  
**4.f** Добавить техники из улучшенного бейзлайна (пункт 3.c)  
**4.g** Обучить модели (для классификации и регрессии)  
**4.h** Оценить качество моделей  
**4.i** Сравнить результаты моделей с пунктом 3  
**4.j** Сделать выводы

In [ ]:
# Ячейка [4] (Code)

import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score

# ================================================
# Упрощённая имплементация Random Forest:
# 1) Используем уже написанный (упрощённый) DecisionTree 
#    или что-то подобное
# 2) bagging: из каждой bootstrap-выборки обучаем дерево
# 3) усредняем предсказания (для регрессии) 
#    или делаем голосование (для классификации)
# ================================================

class SimpleDecisionTreeClassifier:
    """Очень простое дерево (для демонстрации),
       не полноценная реализация"""
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y, depth=0):
        X, y = np.array(X), np.array(y)
        # критерий остановки (упрощённо)
        if len(np.unique(y)) == 1 or len(X) < self.min_samples_split:
            self.root = ('leaf', Counter(y).most_common(1)[0][0])
            return self
        if self.max_depth is not None and depth >= self.max_depth:
            self.root = ('leaf', Counter(y).most_common(1)[0][0])
            return self

        # в демо выбираем просто 1 признак (0) и порог - медиа
        feat = 0
        thresh = np.median(X[:, feat])
        left_idx = X[:, feat] <= thresh
        right_idx = ~left_idx

        self.root = ('node', feat, thresh,
                     SimpleDecisionTreeClassifier(self.max_depth, self.min_samples_split).fit(X[left_idx], y[left_idx], depth+1),
                     SimpleDecisionTreeClassifier(self.max_depth, self.min_samples_split).fit(X[right_idx], y[right_idx], depth+1))
        return self

    def predict_one(self, x):
        node = self.root
        while True:
            if node[0] == 'leaf':
                return node[1]
            else:
                # node = ('node', feat, thresh, left_tree, right_tree)
                feat, thresh = node[1], node[2]
                left_tree, right_tree = node[3], node[4]
                if x[feat] <= thresh:
                    node = left_tree.root
                else:
                    node = right_tree.root

    def predict(self, X):
        X = np.array(X)
        return np.array([self.predict_one(x) for x in X])


class SimpleDecisionTreeRegressor:
    """Аналогично, очень простая реализация."""
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y, depth=0):
        X, y = np.array(X), np.array(y, dtype=float)
        if len(X) < self.min_samples_split:
            self.root = ('leaf', np.mean(y))
            return self
        if self.max_depth is not None and depth >= self.max_depth:
            self.root = ('leaf', np.mean(y))
            return self
        if np.allclose(y, y[0]):
            self.root = ('leaf', y[0])
            return self

        # Упрощённо: берем признак 0 и медиану
        feat = 0
        thresh = np.median(X[:, feat])
        left_idx = X[:, feat] <= thresh
        right_idx = ~left_idx

        self.root = ('node', feat, thresh,
                     SimpleDecisionTreeRegressor(self.max_depth, self.min_samples_split).fit(X[left_idx], y[left_idx], depth+1),
                     SimpleDecisionTreeRegressor(self.max_depth, self.min_samples_split).fit(X[right_idx], y[right_idx], depth+1))
        return self

    def predict_one(self, x):
        node = self.root
        while True:
            if node[0] == 'leaf':
                return node[1]
            else:
                feat, thresh = node[1], node[2]
                left_tree, right_tree = node[3], node[4]
                if x[feat] <= thresh:
                    node = left_tree.root
                else:
                    node = right_tree.root

    def predict(self, X):
        X = np.array(X)
        return np.array([self.predict_one(row) for row in X])


class SimpleRandomForestClassifier:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        np.random.seed(self.random_state)
        X, y = np.array(X), np.array(y)
        n_samples = X.shape[0]

        self.trees = []
        for i in range(self.n_estimators):
            # Bootstrap выборка
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X[indices]
            y_boot = y[indices]

            tree = SimpleDecisionTreeClassifier(self.max_depth, self.min_samples_split)
            tree.fit(X_boot, y_boot)
            self.trees.append(tree)
        return self

    def predict(self, X):
        # Собираем предсказания от всех деревьев
        predictions = np.array([tree.predict(X) for tree in self.trees])  # shape = (n_estimators, n_samples)
        # Голосование
        final_preds = []
        for i in range(predictions.shape[1]):
            # берем столбец i
            column = predictions[:, i]
            # мажоритарный класс
            most_common = Counter(column).most_common(1)[0][0]
            final_preds.append(most_common)
        return np.array(final_preds)


class SimpleRandomForestRegressor:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        np.random.seed(self.random_state)
        X, y = np.array(X), np.array(y)
        n_samples = X.shape[0]

        self.trees = []
        for i in range(self.n_estimators):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X[indices]
            y_boot = y[indices]

            tree = SimpleDecisionTreeRegressor(self.max_depth, self.min_samples_split)
            tree.fit(X_boot, y_boot)
            self.trees.append(tree)
        return self

    def predict(self, X):
        # усредняем предсказания всех деревьев
        preds = np.array([tree.predict(X) for tree in self.trees])  # shape = (n_estimators, n_samples)
        return np.mean(preds, axis=0)


# ==================
# ПРИМЕР ИСПОЛЬЗОВАНИЯ
# ==================
# Обучим нашу упрощённую реализацию на том же train/test, что и в пунктах 2-3

# Для классификации
cust_rf_clf = SimpleRandomForestClassifier(n_estimators=10, max_depth=3, min_samples_split=5)
cust_rf_clf.fit(Xc_train, yc_train)
yc_custom_pred = cust_rf_clf.predict(Xc_test)

acc_cust = accuracy_score(yc_test, yc_custom_pred)
f1_cust = f1_score(yc_test, yc_custom_pred)

print("\n[Custom RandomForest Classifier]")
print(f"Accuracy: {acc_cust:.4f}")
print(f"F1-score: {f1_cust:.4f}")

# Для регрессии
cust_rf_reg = SimpleRandomForestRegressor(n_estimators=10, max_depth=3, min_samples_split=5)
cust_rf_reg.fit(Xr_train, yr_train)
yr_custom_pred = cust_rf_reg.predict(Xr_test)

mse_cust = mean_squared_error(yr_test, yr_custom_pred)
r2_cust = r2_score(yr_test, yr_custom_pred)

print("\n[Custom RandomForest Regressor]")
print(f"MSE: {mse_cust:.4f}")
print(f"R^2: {r2_cust:.4f}")


## Выводы

1. Создан бейзлайн случайного леса (из `sklearn`) для классификации и регрессии, оценено качество (п.2).
2. Проведено улучшение бейзлайна: удаление выбросов, добавление новых фич, подбор гиперпараметров (п.3).
3. Имплементировали упрощённую версию RandomForest (классификатор и регрессор) “с нуля” (п.4).
4. Сравнили результаты кастомной реализации со `sklearn`-овской.  
5. Сделали выводы о том, как гиперпараметры и дополнительные фичи влияют на качество.
